In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id="microsoft/DialoGPT-medium"
device="cuda"
tokenizer=AutoTokenizer.from_pretrained(model_id, padding_side="right")
if tokenizer.pad_token is None:
  tokenizer.pad_token=tokenizer.eos_token
model=AutoModelForCausalLM.from_pretrained(model_id)
model=model.to(device)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='microsoft/DialoGPT-medium', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
from datasets import Dataset
import pandas as pd
df=pd.read_csv("processed_dataset_with_index.csv", index_col=0).head(10000)

In [ ]:
df.shape, df.memory_usage()

((10000, 3),
 Index          80000
 response       80000
 input          80000
 index_level    80000
 dtype: int64)

In [ ]:
conv=Dataset.from_pandas(df)
conv

Dataset({
    features: ['response', 'input', 'index_level', 'context'],
    num_rows: 10000
})

In [ ]:
conv[0]

{'response': 'Was this a friend you were in love with, or just a best friend?',
 'input': 'I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world. Was this a friend you were in love with, or just a best friend?',
 'index_level': 0,
 'context': 'I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.'}

In [ ]:
def chat(inp=None):
  model_cpu=model.to("cpu")
  for step in range(5):
    if not inp:
      new_user_input_ids=tokenizer.encode(input(">> User:")+ tokenizer.eos_token, return_tensors="pt")
    else:
      print(">> User: ", inp[step])
      new_user_input_ids=tokenizer.encode(inp[step]+ tokenizer.eos_token, return_tensors="pt")
    bot_input_ids=torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    chat_history_ids=model_cpu.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


In [ ]:
chat()

>> User:hey


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


DialoGPT: Hey, how's it going?
>> User:great, what about you
DialoGPT: I'm good, how about you?
>> User:good
DialoGPT: Good, I'm glad to hear that.
>> User:i feel lonely
DialoGPT: I feel that
>> User:how
DialoGPT: I feel that


In [ ]:
def encode(examples):
  encoded=tokenizer(examples["input"], truncation=True, padding="max_length", max_length=40, return_tensors="pt")
  encoded["labels"]=encoded["input_ids"][:]
  return encoded

In [ ]:
split_data=conv.train_test_split(test_size=0.2)
encoded_data=split_data.map(encode, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['response', 'input', 'index_level', 'context', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['response', 'input', 'index_level', 'context', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args=TrainingArguments(
    output_dir="fine_tuned_dialogpt",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    learning_rate=2e-5,
    fp16=True,
    report_to="none"
)

In [ ]:
training_args.device

device(type='cuda', index=0)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data["train"],
    eval_dataset=encoded_data["test"]
)

In [ ]:
pre_eval_results=trainer.evaluate(encoded_data["test"])
pre_val_predictions=trainer.predict(encoded_data["test"].select(range(10)))

In [ ]:
print(pre_eval_results)

{'eval_loss': 6.4392852783203125, 'eval_model_preparation_time': 0.0054, 'eval_runtime': 27.0888, 'eval_samples_per_second': 73.831, 'eval_steps_per_second': 36.916}


In [ ]:
import numpy as np
def generate_response(predictions):
  responses=[]
  for idx, pred in enumerate(predictions):
    response=tokenizer.decode(np.argmax(pred, axis=-1), skip_special_tokens=True)
    responses.append(response)
  return responses

In [ ]:
res=generate_response(pre_val_predictions.predictions)
res

[" the's't a.. but'm they fix not too bad.I,,'s't mean like bad. I'm you's the help his..IIIIIIII",
 " andyes'm the new.I do it live?SheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheShe",
 " the, you job!Thanks, I'm aIirl to. in. was't even it a few.IIIIIIIIIIIIIIIII",
 " and's like a good idea.to you's't get. a..thanksa, weathers are not pretty.Theed.. the.ThanksThanksThanksThanksThanksThanksThanksThanks",
 " the is a many.I'm if it's a to the than than less people. the countryer, it's beenI'm it's not case ofIIIIIIII",
 " and behe'm sure on you toHeive's means no sense. that said it are a you one to be homelessiveIIIIIII",
 " and a anded.It you see theible.It wait,'m like the.latablecials. but to see more to car works.IIIIIIIIII",
 " and in I was abroad lot of my exam part.I I I,'m in much a about my grades score.I's is a good of work.IIIIIII",
 " and I's a to avoid careful little player. not your from far as possible. you are no many people people. th

In [ ]:
inp=encoded_data["test"].select(range(10))
inp["input"]

["Oh that can be annoying sometimes, I hope they're not too serious. Oh well that doesn't sound too bad, I hope he gets some in return though.",
 'Yes. I love their music. Where does she work?',
 "Well done, great job. Yes, I am.  About 3 months ago I couldn't run for a minute.",
 "Ah that sounds like a good time. Hope it doesn't rain for you guys. Yea the radars are usually accurate. Cook good food for me.",
 "My yard has too actually. I wonder if it's due to more sun and more rain across the board Yes, it has. I know that's the source.",
 'He better, I am counting on him. lol  that literally makes no sense like i get it you are homeless but no reason to be dirty! ugh ',
 'Nothing like redneck engineering. Did you use Flex tape. Oh darn I really like those infomercials, wanted to know how the stuff worked.',
 'Back in school, I studied a lot for an important exam. But, still I was very much worried about my test results. that sure is a lot of savings',
 'Yes.. it is best to be a defen

In [ ]:
trainer.train()

Step,Training Loss
500,2.725400
1000,2.466400
1500,2.422900
2000,2.371000
2500,2.156000
3000,2.126200
3500,2.129400
4000,2.127800
4500,1.955100
5000,1.930600


TrainOutput(global_step=10000, training_loss=2.0127870300292967, metrics={'train_runtime': 3237.291, 'train_samples_per_second': 12.356, 'train_steps_per_second': 3.089, 'total_flos': 2902189670400000.0, 'train_loss': 2.0127870300292967, 'epoch': 5.0})

In [ ]:
post_eval_results=trainer.evaluate(encoded_data["test"])
post_val_predictions=trainer.predict(encoded_data["test"].select(range(10)))

In [ ]:
post_eval_results

{'eval_loss': 2.113478899002075,
 'eval_model_preparation_time': 0.0054,
 'eval_runtime': 37.9262,
 'eval_samples_per_second': 52.734,
 'eval_steps_per_second': 26.367,
 'epoch': 5.0}

In [ ]:
res=generate_response(pre_val_predictions.predictions)
res

[" the's't a.. but'm they fix not too bad.I,,'s't mean like bad. I'm you's the help his..IIIIIIII",
 " andyes'm the new.I do it live?SheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheSheShe",
 " the, you job!Thanks, I'm aIirl to. in. was't even it a few.IIIIIIIIIIIIIIIII",
 " and's like a good idea.to you's't get. a..thanksa, weathers are not pretty.Theed.. the.ThanksThanksThanksThanksThanksThanksThanksThanks",
 " the is a many.I'm if it's a to the than than less people. the countryer, it's beenI'm it's not case ofIIIIIIII",
 " and behe'm sure on you toHeive's means no sense. that said it are a you one to be homelessiveIIIIIII",
 " and a anded.It you see theible.It wait,'m like the.latablecials. but to see more to car works.IIIIIIIIII",
 " and in I was abroad lot of my exam part.I I I,'m in much a about my grades score.I's is a good of work.IIIIIII",
 " and I's a to avoid careful little player. not your from far as possible. you are no many people people. th

In [ ]:
!ls fine_tuned_dialogpt/checkpoint-10000


config.json		model.safetensors  rng_state.pth  trainer_state.json
generation_config.json	optimizer.pt	   scheduler.pt   training_args.bin


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Checkpoint'ten modeli yükleyin
model_checkpoint = "fine_tuned_dialogpt/checkpoint-8500"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Varsayılan tokenizer'ı yükleyin
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Model ve tokenizer'ı test edin
def chat_with_model(user_input):
    inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
    outputs = model.generate(inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)

response = chat_with_model("i feel lonely")
print("Chatbot:", response)


Chatbot: I feel you there. I have a lot of friends who are also single.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Checkpoint yolunu belirtin
model_checkpoint = "fine_tuned_dialogpt/checkpoint-8500"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")  # Varsayılan tokenizer'ı alın

# Yeni bir klasöre modeli ve tokenizer'ı kaydedin
output_dir = "./saved_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/merges.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
def interactive_chat():
    from transformers import AutoTokenizer, AutoModelForCausalLM

    # Model ve tokenizer'ı yükleme
    model_dir = "fine_tuned_dialogpt/checkpoint-10000"
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained(model_dir)

    # Konuşma geçmişini tutmak için bir liste
    chat_history = []

    print("Chatbot ile konuşmaya başlayabilirsiniz. Çıkmak için 'çık' yazın.")

    while True:
        # Kullanıcı girdisi
        user_input = input("Siz: ")
        if user_input.lower() == "çık":
            print("Chatbot: Görüşmek üzere!")
            break

        # Kullanıcı girdisini geçmişe ekleme
        chat_history.append(user_input)

        # Sohbet geçmişini düz bir metin olarak oluşturma
        chat_history_text = ""
        for i, message in enumerate(chat_history):
            if i % 2 == 0:  # Kullanıcı mesajı
                chat_history_text += f"[USER]: {message}\n"
            else:  # Bot mesajı
                chat_history_text += f"[BOT]: {message}\n"

        # Girdi metnini hazırlama
        input_text = chat_history_text + f"[USER]: {user_input}\n[BOT]:"
        inputs = tokenizer.encode(input_text, return_tensors="pt")

        # Yanıt üretme
        outputs = model.generate(
            inputs,
            max_length=300,  # Yanıt uzunluğu
            max_new_tokens=50,
            temperature=0.8,  # Yanıtların yaratıcılığı için
            top_p=0.95,  # Çeşitlilik için üst sıralama numaraları
            num_beams=5,  # Beam search ile daha anlamlı yanıtlar
            pad_token_id=tokenizer.eos_token_id
        )

        # Yanıtı çözümleme
        response = tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
        print(f"Chatbot: {response}")

        # Bot yanıtını geçmişe ekleme
        chat_history.append(response)


In [ ]:
interactive_chat()

Chatbot ile konuşmaya başlayabilirsiniz. Çıkmak için 'çık' yazın.


Both `max_new_tokens` (=50) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot:  Hello, my name is Richard and I am a computer programing major.


Both `max_new_tokens` (=50) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot:  i feel depressed
